In [1]:
import asyncio
import aiohttp
import async_timeout
from collections import OrderedDict, Iterable
logger = object #<-----gahhhhhhh I know I know...

In [2]:
class YouFuckedUpError(BaseException):
    pass

In [3]:
try:
    import uvloop
    asyncio.set_event_loop_policy(uvloop.EventLoopPolicy())
except ImportError:
    pass

In [4]:
async def fetch(url, headers, session, semaphore):
    with (await semaphore): # <--...UH...K?
        # Can't think of when this if/else bloc would be necessary, 
        # but the herd compells me:
        if callable(headers):
            _headers = headers()
        else:
            _headers = headers
        try:
            async with session.get(url, headers=_headers) as response:
                if response.status in (200, 201):
                    data = await response.text()
                    return data
                else:
                    error_code = response.status
                    raise aiohttp.ClientResponseError()
        except aiohttp.ClientResponseError as err:
            print('[FETCH-ERROR] url: {} -- code: {}\t msg: {}'.format(url, error_code, (err.msg if err.msg else "NO DATA FROM SERVER")))
            return None

In [8]:
# # Test `fetch()`
# class bucket:
#     bucket = []
#     @classmethod
#     def submit(cls, obj):
#         cls.bucket.append(obj)
#         print('object saved in bucket.')
#     @classmethod    
#     async def do(cls, url, headers, session, semaphore):
#         data = await fetch(url, headers, session, semaphore)
#         cls.submit(data)
    
# loop = asyncio.get_event_loop()
# url = 'https://httpbin.org/get'
# url2 = 'https://httpbin.org/uuid'
# sem = asyncio.Semaphore(2)
# headers = {'Accept' : '*/*'}

# with aiohttp.ClientSession() as session:
#     task_1 = asyncio.ensure_future(bucket.do(url2, headers, session, sem))
#     task_2 = asyncio.ensure_future(bucket.do(url, headers, session, sem))
#     loop.run_until_complete(asyncio.gather(task_1, task_2))
# from pprint import pprint
# pprint(bucket.bucket)
# # Totes works.

Creating a client session outside of coroutine
client_session: <aiohttp.client.ClientSession object at 0x000001639EE67588>


object saved in bucket.
object saved in bucket.
['{\n  "uuid": "a263c6ca-6595-4b22-8378-e29075baf13c"\n}\n',
 '{\n'
 '  "args": {}, \n'
 '  "headers": {\n'
 '    "Accept": "*/*", \n'
 '    "Accept-Encoding": "gzip, deflate", \n'
 '    "Connection": "close", \n'
 '    "Host": "httpbin.org", \n'
 '    "User-Agent": "Python/3.6 aiohttp/2.2.5"\n'
 '  }, \n'
 '  "origin": "64.128.89.162", \n'
 '  "url": "https://httpbin.org/get"\n'
 '}\n']


In [15]:
class Manifest:
    """IMPORTANT NOTE: Manifest() & Orchestrator() ARE *EXTREMELY-EXTREMELY-EXTREMELY* COUPLED.
        MOAR NOTES: 
         - This class exists SOLELY to abstract input from the module's mechanics.
         - ONE Orchestrator() can handle MANY Manifest()s
        """
    def __init__(self, start_url='https://httpbin.org/get', headers=None, depth=3):
        self.start_url = start_url
        if headers:
            self.headers = headers
        else:
            self.headers = {}
        self.depth = depth
        
        self.base_url = None
        self.urls_count = 0
        
        


In [28]:
class Orchestrator(object):
    """IMPORTANT NOTE: Manifest() & Orchestrator() ARE *EXTREMELY-EXTREMELY-EXTREMELY* COUPLED."""
    
    #TODO: This class should eventually be able to take a loop as an arg. For now only default is allowed.
    #TODO: The above todo will allow a MANY to MANY relationship b/w Orchestrator() & Manifest().
    _loop = None
    _semaphore = None
    _error_urls = []
    _manifests = []
    _queue = None
    
    def __init__(self, concurency=5, loop=None, queue_len=0):
        if loop == None:
            if Orchestrator._loop == None:
                Orchestrator._loop = asyncio.get_event_loop()
            if Orchestrator._queue == None:
                Orchestrator._queue = asyncio.Queue(maxsize=queue_len, loop=Orchestrator._loop)
            if Orchestrator._semaphore == None:
                Orchestrator._semaphore = asyncio.Semaphore(concurency)
        else:
            raise YouFuckedUpError("Can't do that yet.")
            exit(-1)
    
    @classmethod
    def _is_ready(cls):
        try:
            assert cls._loop is not None
            assert cls._semaphore is not None
            assert cls._queue is not None
            return True
        except AssertionError as err:
            return False
        
    @classmethod
    async def _run(cls):
        if cls._is_ready():
            try:
                cls._loop.run_forever(dump_loop)
            except KeyboardInterrupt:
                print('mission complete?')
        else:
            print('Orchestrator._is_ready() failed')
    
    @classmethod
    async def dump_loop(cls):
        while True:
            stuff_i_guess = await cls._queue.get()
            print(stuff_i_guess)

        
    @classmethod
    async def load(cls, manifest):
        assert isinstance(manifest, Manifest), "There's a special class for this. Ya need to use it."
        async with aiohttp.ClientSession(loop=cls._loop) as session:
            task = asyncio.ensure_future(fetch(manifest.url, manifest.headers, session, cls._semaphore))
            await cls._queue.put(task)
        
            
        
        
        
        
            
    

In [31]:
m = Manifest()
m2 = Manifest(start_url='https://httpbin.org/ip')
o = Orchestrator()
o.load(m)
o.load(m2)
o._run()

C:\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: coroutine 'Orchestrator.load' was never awaited
  after removing the cwd from sys.path.
C:\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: coroutine 'Orchestrator.load' was never awaited
  """


TypeError: 'coroutine' object is not callable

In [25]:
print('hi')

hi
